In [1]:
import pandas as pd
import numpy as np
import datetime

# 주가변동에 따른 뉴스기사 labeling

In [2]:
# 주식장 오픈 날짜 데이터 불러오기
date_df = pd.read_csv('../../../Stock_Data/Data/stock_df/date_list.csv')
del date_df['Unnamed: 0']
del date_df['article']
date_df = date_df[date_df['Date'] <= '2019-12-31']

### 여러개 종목 labeling

In [4]:
# 데이터 불러오기, 전처리
kospis = ['GS건설','KB금융','아모레퍼시픽','삼성전자','이마트'] 

for kospi in kospis:
   
    # 주가 데이터 불러오기
    stock_df = pd.read_csv('../../../Stock_Data/Data/stock_df/stock_df/{}_stock.csv'.format(kospi))
    change_df = stock_df[['Date','Close']]
    # 주가 변동률 컬럼 생성
    rocs = []
    for i in range(len(change_df)):
        if i != 0:
            yesterday = change_df['Close'][(i-1)]
            today = change_df['Close'][i]
            roc = (today-yesterday)/yesterday
            rocs.append(roc)
        else:
            rocs.append(0)
    change_df['RateofChange'] = rocs
    # 변동률 < 0: -1, 변동률 > 0: 1로 인코딩
    change_df['Label'] = np.zeros(len(change_df),)
    change_df['Label'].loc[change_df['RateofChange'] < 0] = -1
    change_df['Label'].loc[change_df['RateofChange'] > 0] = 1
    # 첫날(2014/01/02)는 전날 데이터가 없으므로 다음날과 동일하게 설정
    change_df['Label'][0] = change_df['Label'][1]
    # 변동률이 0일 경우, 전날 변동 추이 반영
    for i in range(len(change_df)):
        if change_df['Label'][i]==0:
            change_df['Label'][i] = change_df['Label'][(i-1)]

    # 기사데이터 불러오기
    article_df = pd.read_csv('../../../Crawling/Data/kospi20/{}_df.csv'.format(kospi))
    article_df = article_df.drop_duplicates('Title',keep = 'first')
    article_df = article_df.drop(article_df[article_df['Date'] == '2019.12.31'].index)
    article_df = article_df.dropna(axis=0)
    article_df = article_df.reset_index(drop=True)
    for i in range(len(article_df)):
        article_df['Date'][i] = article_df['Date'][i].replace('.','-')
    del article_df['Unnamed: 0']
    # 휴장 날짜 변경
    while True:
        index = article_df['Date'].isin(date_df['Date']) == False
        holidays = article_df['Date'][index]
        if len(holidays) == 0:
            break
        else:
            for i in range(len(holidays)):
                holiday = holidays.iloc[i]
                conv_hd = datetime.datetime.strptime(holiday,'%Y-%m-%d')
                not_holiday = str(conv_hd + datetime.timedelta(days=1))[:10]
                holidays.iloc[i] = not_holiday
            article_df['Date'][index] = holidays 
    
    # 기사와 변동률 테이블 조인 후 내보내기
    merge_df = pd.merge(change_df, article_df, left_on=['Date'], right_on=['Date'], how='right')
    merge_df.to_csv('../../Data/PriceChange/{}_roc_traindata.csv'.format(kospi))

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

### 한종목 labeling

In [88]:
kospi = ['CJ제일제당']

# 주가 데이터 불러오기
stock_df = pd.read_csv('../../../Stock_Data/Data/stock_df/stock_df/{}_stock.csv'.format(kospi))
change_df = stock_df[['Date','Close']]
# 주가 변동률 컬럼 생성
rocs = []
for i in range(len(change_df)):
    if i != 0:
        yesterday = change_df['Close'][(i-1)]
        today = change_df['Close'][i]
        roc = (today-yesterday)/yesterday
        rocs.append(roc)
    else:
        rocs.append(0)
change_df['RateofChange'] = rocs
# 변동률 < 0: -1, 변동률 > 0: 1로 인코딩
change_df['Label'] = np.zeros(len(change_df),)
change_df['Label'].loc[change_df['RateofChange'] < 0] = -1
change_df['Label'].loc[change_df['RateofChange'] > 0] = 1
# 첫날(2014/01/02)는 전날 데이터가 없으므로 다음날과 동일하게 설정
change_df['Label'][0] = change_df['Label'][1]
# 변동률이 0일 경우, 전날 변동 추이 반영
for i in range(len(change_df)):
    if change_df['Label'][i]==0:
        change_df['Label'][i] = change_df['Label'][(i-1)]

# 기사데이터 불러오기
article_df = pd.read_csv('../../../Crawling/Data/kospi20/{}_df.csv'.format(kospi))
article_df = article_df.drop_duplicates('Title',keep = 'first')
article_df = article_df.drop(article_df[article_df['Date'] == '2019.12.31'].index)
article_df = article_df.dropna(axis=0)
article_df = article_df.reset_index(drop=True)
for i in range(len(article_df)):
    article_df['Date'][i] = article_df['Date'][i].replace('.','-')
del article_df['Unnamed: 0']
# 휴장 날짜 변경
while True:
    index = article_df['Date'].isin(date_df['Date']) == False
    holidays = article_df['Date'][index]
    if len(holidays) == 0:
        break
    else:
        for i in range(len(holidays)):
            holiday = holidays.iloc[i]
            conv_hd = datetime.datetime.strptime(holiday,'%Y-%m-%d')
            not_holiday = str(conv_hd + datetime.timedelta(days=1))[:10]
            holidays.iloc[i] = not_holiday
        article_df['Date'][index] = holidays 

# 기사와 변동률 테이블 조인 후 내보내기
merge_df = pd.merge(change_df, article_df, left_on=['Date'], right_on=['Date'], how='right')
merge_df.to_csv('../../Data/PriceChange/{}_roc_traindata.csv'.format(kospi))

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
